In [1]:
from attack_func import *
from pathlib import Path
import pandas as pd
import torch

In [2]:
datapath = Path('data')
df = pd.read_csv('..'/datapath/"SGCC_data"/"data.csv")
df = df.set_index('FLAG').drop('CONS_NO', axis=1)
df.columns = pd.to_datetime(df.columns)
df = df.sort_index(axis=1)
df.shape

(42372, 1034)

In [3]:
groups = df.groupby(df.index, sort=False)
group_normal = groups.get_group(0)
group_attack = groups.get_group(1)
group_normal.shape, group_attack.shape

((38757, 1034), (3615, 1034))

In [4]:
threshold = 5  # 设置阈值，根据实际情况调整
group_normal = group_normal.dropna(thresh=df.shape[1]-threshold)
# group_normal = group_normal.interpolate(method='linear', axis=1)
group_normal.shape

(4985, 1034)

In [5]:
torch.isnan(torch.tensor(group_normal.values)).any()

tensor(True)

In [6]:
group_normal = group_normal.interpolate(method='linear', axis=1).fillna(0)
group_normal.shape

(4985, 1034)

In [7]:
# nan_rows = group_normal[group_normal.isnull().any(axis=1)]
# print(nan_rows)

In [8]:
torch.isnan(torch.tensor(group_normal.values)).any()

tensor(False)

In [9]:
group_normal_2016 = group_normal.loc[:, (group_normal.columns.year == 2016)].reset_index(drop=True)
group_normal_2016

,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,...,2016-10-22,2016-10-23,2016-10-24,2016-10-25,2016-10-26,2016-10-27,2016-10-28,2016-10-29,2016-10-30,2016-10-31
0,8.72,3.95,4.08,4.42,4.50,7.26,5.54,4.56,5.26,4.75,...,1.95,2.06,1.96,1.99,1.92,1.82,1.89,2.06,2.63,1.96
1,6.60,5.89,4.72,4.45,6.88,5.09,4.18,7.24,4.60,5.00,...,8.25,9.96,12.59,9.64,10.42,14.07,7.62,8.37,5.36,6.53
2,12.58,6.47,0.51,0.13,0.11,0.11,0.07,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,14.67,10.10,16.15,10.80,10.32,13.13,15.75,14.01,9.70,13.22,...,6.94,6.71,6.07,9.99,6.66,10.40,9.77,5.50,9.37,7.04
4,22.75,18.07,22.25,20.85,19.46,19.13,19.92,19.57,15.56,21.29,...,13.51,11.94,10.16,11.12,15.77,12.74,14.41,12.35,15.25,10.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4980,6.03,4.92,6.11,5.46,5.22,6.04,4.77,4.67,6.17,4.41,...,13.46,11.02,8.41,7.99,10.11,12.41,11.84,11.08,6.55,5.60
4981,0.01,0.03,0.10,0.01,0.03,0.00,0.03,0.00,0.02,0.07,...,0.04,0.14,0.04,0.16,0.14,0.04,0.11,0.03,0.03,0.16
4982,10.72,10.35,9.07,8.66,10.23,8.34,6.18,9.85,8.94,8.35,...,10.31,8.34,9.67,6.67,9.65,8.01,7.68,9.29,10.31,8.55
4983,18.70,16.26,17.91,13.26,12.63,10.50,10.01,14.24,12.76,12.96,...,23.93,33.49,30.00,29.86,32.41,35.18,38.41,26.18,30.13,33.64


In [10]:
# 为每个用户生成两个0到1之间的随机数
start_rates = np.random.rand(len(group_normal_2016))
end_rates = np.random.rand(len(group_normal_2016))
# 确保 end_rates 对应的位置上的数字比 start_rates 上的大
mask = end_rates < start_rates
start_rates[mask], end_rates[mask] = end_rates[mask], start_rates[mask]

In [11]:
# 创建一个新的DataFrame来存储攻击的结果
attack_df = group_normal_2016.copy()
# 创建一个新的DataFrame来存储标签
label_df = pd.DataFrame(0, index=group_normal_2016.index, columns=group_normal_2016.columns)

In [12]:
print(f"Time length: {len(group_normal_2016.columns)}")
# 对每个用户施加攻击
for i in range(len(group_normal_2016)):
    # 计算开始索引和结束索引
    start_index = int(start_rates[i] * len(group_normal_2016.columns))
    end_index = int(end_rates[i] * len(group_normal_2016.columns))

    # 计算持续时间
    duration = end_index - start_index

    # 将原始的DataFrame转换为数组
    data_array = np.expand_dims(group_normal_2016.iloc[i].values, axis=0)

    # 在数组上施加攻击
    attack_result = type1_attack(np.expand_dims(data_array, axis=1), start_index, duration)

    print(f"{start_index} - {end_index} : {duration}")

    # 将攻击结果存储到attack_df中
    attack_df.iloc[i, :] = attack_result.squeeze()

    # 更新标签
    label_df.iloc[i, start_index:end_index] = 1

Time length: 304
170 - 228 : 58
13 - 195 : 182
152 - 194 : 42
116 - 129 : 13
72 - 108 : 36
27 - 222 : 195
15 - 68 : 53
23 - 89 : 66
3 - 303 : 300
14 - 131 : 117
196 - 290 : 94
91 - 187 : 96
143 - 188 : 45
210 - 217 : 7
84 - 188 : 104
20 - 291 : 271
59 - 106 : 47
122 - 170 : 48
22 - 156 : 134
115 - 273 : 158
36 - 271 : 235
82 - 178 : 96
199 - 237 : 38
37 - 175 : 138
48 - 265 : 217
103 - 216 : 113
224 - 257 : 33
172 - 195 : 23
38 - 265 : 227
47 - 294 : 247
227 - 290 : 63
10 - 17 : 7
201 - 236 : 35
150 - 168 : 18
218 - 288 : 70
62 - 200 : 138
41 - 141 : 100
218 - 271 : 53
45 - 269 : 224
167 - 243 : 76
116 - 156 : 40
21 - 33 : 12
140 - 147 : 7
58 - 115 : 57
221 - 247 : 26
227 - 289 : 62
76 - 155 : 79
195 - 253 : 58
175 - 223 : 48
81 - 284 : 203
8 - 196 : 188
179 - 197 : 18
34 - 262 : 228
47 - 260 : 213
132 - 142 : 10
68 - 214 : 146
221 - 231 : 10
161 - 251 : 90
100 - 179 : 79
93 - 220 : 127
79 - 288 : 209
168 - 190 : 22
7 - 82 : 75
131 - 249 : 118
82 - 141 : 59
299 - 301 : 2
30 - 137 : 107

In [13]:
# 将攻击的结果和标签存储为两个CSV文件
attack_df.to_csv('attack.csv', index=False)
label_df.to_csv('label.csv', index=False)